In [4]:
# Install ifcopenshell for IFC parsing
!pip install ifcopenshell matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 MB 17.7 MB/s eta 0:00:00


Cloning Repo - Depending on IFC File required

In [6]:
!git clone https://github.com/sylvainHellin/ifc-bench.git

Cloning into 'ifc-bench'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 90 (delta 1), reused 6 (delta 0), pack-reused 78 (from 1)
Receiving objects: 100% (90/90), 45.07 MiB | 30.23 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Filtering content: 100% (5/5), 163.22 MiB | 62.23 MiB/s, done.


# Tool A (Useful Surface Area)

In [ ]:

# Requirements
import math
import json
import ifcopenshell
import ifcopenshell.geom

# --------------------------
# Geometry settings
# --------------------------
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

# --------------------------
# IFC helpers
# --------------------------
def calculate_space_area(space):
    """Approximate area by summing triangle areas from the space mesh."""
    try:
        shape = ifcopenshell.geom.create_shape(settings, space)
        verts = shape.geometry.verts
        faces = shape.geometry.faces
        area = 0.0

        for i in range(0, len(faces), 3):
            i0, i1, i2 = faces[i] * 3, faces[i + 1] * 3, faces[i + 2] * 3
            v0 = verts[i0:i0 + 3]
            v1 = verts[i1:i1 + 3]
            v2 = verts[i2:i2 + 3]

            a = math.sqrt((v1[0]-v0[0])**2 + (v1[1]-v0[1])**2 + (v1[2]-v0[2])**2)
            b = math.sqrt((v2[0]-v1[0])**2 + (v2[1]-v1[1])**2 + (v2[2]-v1[2])**2)
            c = math.sqrt((v0[0]-v2[0])**2 + (v0[1]-v2[1])**2 + (v0[2]-v2[2])**2)

            s = (a + b + c) / 2.0
            area += math.sqrt(max(s * (s - a) * (s - b) * (s - c), 0.0))

        return float(area)
    except Exception:
        return 0.0

def dwelling_area_check(ifc_model_path, min_area=36.0):
    """Check if total dwelling useful area >= min_area."""
    model = ifcopenshell.open(ifc_model_path)
    spaces = model.by_type("IfcSpace")

    total_area = 0.0
    room_areas = {}

    for space in spaces:
        area = calculate_space_area(space)
        room_name = getattr(space, "LongName", None) or getattr(space, "Name", None) or "Unnamed"
        room_areas[str(room_name)] = float(area)
        total_area += float(area)

    if total_area >= float(min_area):
        return {
            "result": "pass",
            "reason": f"Total useful area {total_area:.2f} m² >= {float(min_area):.2f} m²",
            "total_area": float(total_area),
            "room_areas": room_areas,
        }

    return {
        "result": "fail",
        "reason": f"Total useful area {total_area:.2f} m² < {float(min_area):.2f} m²",
        "total_area": float(total_area),
        "room_areas": room_areas,
    }

# --------------------------
# Tool entrypoint (what your LLM router will call later)
# --------------------------
def dwelling_area_check_tool(ifc_model_path: str, min_area: float = 36.0):
    return dwelling_area_check(ifc_model_path, min_area)

# --------------------------
# Schema (API key not needed)
# Plain JSON-schema-like dict, easy to serialize + pass to any LLM framework
# --------------------------
DWELLING_AREA_CHECK_SCHEMA = {
    "name": "dwelling_area_check_tool",
    "description": "Checks whether an IFC dwelling meets a minimum total useful area requirement and returns a room-by-room breakdown.",
    "parameters": {
        "type": "object",
        "properties": {
            "ifc_model_path": {
                "type": "string",
                "description": "Filesystem path to the IFC model."
            },
            "min_area": {
                "type": "number",
                "description": "Minimum total area in m². Default is 36."
            }
        },
        "required": ["ifc_model_path"]
    }
}

# --------------------------
# Optional: local sanity test (no API usage)
# --------------------------
if __name__ == "__main__":
    print("Schema OK:")
    print(json.dumps(DWELLING_AREA_CHECK_SCHEMA, indent=2))

    ifc_path = "/content/ifc-bench/projects/duplex/arc.ifc"
    result = dwelling_area_check_tool(ifc_path, min_area=36.0)
    print(result["result"])
    print(result["reason"])


Schema OK:
{
  "name": "dwelling_area_check_tool",
  "description": "Checks whether an IFC dwelling meets a minimum total useful area requirement and returns a room-by-room breakdown.",
  "parameters": {
    "type": "object",
    "properties": {
      "ifc_model_path": {
        "type": "string",
        "description": "Filesystem path to the IFC model."
      },
      "min_area": {
        "type": "number",
        "description": "Minimum total area in m\u00b2. Default is 36."
      }
    },
    "required": [
      "ifc_model_path"
    ]
  }
}
pass
Total useful area 1711.01 m² >= 36.00 m²


# TOOL B - Living room Height Regulation

In [7]:
import json
import ifcopenshell
import ifcopenshell.geom

# --------------------------
# Geometry settings
# --------------------------
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

# --------------------------
# Helper functions
# --------------------------
def load_model(ifc_model_path):
    """Load IFC model"""
    return ifcopenshell.open(ifc_model_path)

def get_main_living_areas(ifc_model):
    """
    Return spaces considered main living areas.
    Filtering by common keywords in space name.
    """
    spaces = ifc_model.by_type("IfcSpace")
    main_spaces = []
    for s in spaces:
        if s.Name and any(keyword in s.Name.lower() for keyword in ["living", "bedroom", "hall"]):
            main_spaces.append(s)
    return main_spaces

def get_space_height(space):
    """
    Approximate height of a space from geometry bounding box.
    Uses Z coordinates of all vertices.
    """
    try:
        shape = ifcopenshell.geom.create_shape(settings, space)
        verts = shape.geometry.verts
        if not verts:
            return 0.0
        zs = verts[2::3]
        return float(max(zs) - min(zs))
    except Exception:
        return 0.0

# --------------------------
# Main check function
# --------------------------
def living_area_height_check(ifc_model_path, min_height=2.50):
    """
    Check if all main living areas meet the minimum height requirement.

    Returns a dictionary with:
    - result: 'pass' or 'fail'
    - reason: explanation string
    - room_heights: dictionary of room_name -> calculated height
    """
    model = load_model(ifc_model_path)
    spaces = get_main_living_areas(model)

    room_heights = {}
    for s in spaces:
        height = get_space_height(s)
        room_name = s.Name or "Unnamed"
        room_heights[room_name] = height
        if height < min_height:
            return {
                "result": "fail",
                "reason": f"{room_name} height below {min_height}m",
                "room_heights": room_heights
            }

    return {
        "result": "pass",
        "reason": f"All main living areas meet minimum height {min_height}m",
        "room_heights": room_heights
    }

# --------------------------
# Tool entrypoint (for an LLM router later)
# --------------------------
def living_area_height_check_tool(ifc_model_path: str, min_height: float = 2.50):
    return living_area_height_check(ifc_model_path, min_height)

# --------------------------
# Schema (no API key needed)
# --------------------------
LIVING_AREA_HEIGHT_CHECK_SCHEMA = {
    "name": "living_area_height_check_tool",
    "description": "Checks whether main living areas in an IFC meet a minimum height requirement and returns a room-by-room height breakdown.",
    "parameters": {
        "type": "object",
        "properties": {
            "ifc_model_path": {
                "type": "string",
                "description": "Filesystem path to the IFC model."
            },
            "min_height": {
                "type": "number",
                "description": "Minimum height in meters. Default is 2.50."
            }
        },
        "required": ["ifc_model_path"]
    }
}

# --------------------------
# Usage example (prints schema + pass/fail)
# --------------------------
if __name__ == "__main__":
    print("Schema OK:")
    print(json.dumps(LIVING_AREA_HEIGHT_CHECK_SCHEMA, indent=2))

    ifc_path = "/content/ifc-bench/projects/duplex/arc.ifc"  # change if needed
    check = living_area_height_check_tool(ifc_path, min_height=2.50)

    print("\nCheck result:")
    print(check["result"])
    print(check["reason"])
    print("Room heights:", check["room_heights"])


Schema OK:
{
  "name": "living_area_height_check_tool",
  "description": "Checks whether main living areas in an IFC meet a minimum height requirement and returns a room-by-room height breakdown.",
  "parameters": {
    "type": "object",
    "properties": {
      "ifc_model_path": {
        "type": "string",
        "description": "Filesystem path to the IFC model."
      },
      "min_height": {
        "type": "number",
        "description": "Minimum height in meters. Default is 2.50."
      }
    },
    "required": [
      "ifc_model_path"
    ]
  }
}

Check result:
pass
All main living areas meet minimum height 2.5m
Room heights: {}


# TOOL C - Living Room Area Check

In [8]:
import json
import math
import ifcopenshell
import ifcopenshell.geom

# Optional: only needed if you truly have it available
# from ifcopenshell.util.element import get_psets as _get_psets

# --------------------------
# Geometry settings
# --------------------------
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

# --------------------------
# Helper functions
# --------------------------
def load_model(ifc_model_path):
    return ifcopenshell.open(ifc_model_path)

def calculate_space_area(space):
    """
    Approximate area from mesh triangles.
    Note: this is mesh surface area, not guaranteed to be floor area.
    """
    if hasattr(space, "NetFloorArea") and space.NetFloorArea:
        return float(space.NetFloorArea)

    try:
        shape = ifcopenshell.geom.create_shape(settings, space)
        verts = shape.geometry.verts
        faces = shape.geometry.faces
        area = 0.0

        for i in range(0, len(faces), 3):
            idx0, idx1, idx2 = faces[i] * 3, faces[i + 1] * 3, faces[i + 2] * 3
            v0 = verts[idx0:idx0 + 3]
            v1 = verts[idx1:idx1 + 3]
            v2 = verts[idx2:idx2 + 3]

            a = math.sqrt((v1[0]-v0[0])**2 + (v1[1]-v0[1])**2 + (v1[2]-v0[2])**2)
            b = math.sqrt((v2[0]-v1[0])**2 + (v2[1]-v1[1])**2 + (v2[2]-v1[2])**2)
            c = math.sqrt((v0[0]-v2[0])**2 + (v0[1]-v2[1])**2 + (v0[2]-v2[2])**2)

            s = (a + b + c) / 2.0
            area += math.sqrt(max(s * (s - a) * (s - b) * (s - c), 0.0))

        return float(area)
    except Exception:
        return 0.0

def get_space_name(space):
    """Get descriptive name if available, fallback to LongName/Name/GlobalId."""
    if getattr(space, "LongName", None):
        return str(space.LongName)
    if getattr(space, "Name", None):
        return str(space.Name)

    # If you have ifcopenshell.util.element available, you can enable this block:
    # try:
    #     psets = _get_psets(space)
    #     for props in (psets or {}).values():
    #         for k, v in (props or {}).items():
    #             if "name" in str(k).lower() and isinstance(v, str) and v.strip():
    #                 return v.strip()
    # except Exception:
    #     pass

    return str(getattr(space, "GlobalId", "Unnamed"))

def can_fit_square(area_m2, width=2.4, depth=2.4):
    """Approx check: area must allow a width x depth square."""
    return float(area_m2) >= float(width) * float(depth)

# --------------------------
# Living room compliance
# --------------------------
def living_room_compliance(ifc_model_path):
    """
    Rule encoded:
      - Living room min area 10 m²
      - If living+kitchen combined: min area 14 m²
      - Must allow 2.40 x 2.40 m clearance (approx via area check)
    Returns list of per-space results.
    """
    model = load_model(ifc_model_path)
    spaces = model.by_type("IfcSpace")

    results = []

    for space in spaces:
        raw_name = get_space_name(space)
        name = raw_name.lower()
        area = calculate_space_area(space)
        if area <= 0:
            continue

        if "living" in name:
            has_kitchen = "kitchen" in name
            min_area = 14.0 if has_kitchen else 10.0

            clearance_ok = can_fit_square(area, 2.4, 2.4)
            passed = (area >= min_area) and clearance_ok

            reasons = []
            if area < min_area:
                reasons.append(f"Area {area:.2f} m² < required {min_area:.2f} m²")
            if not clearance_ok:
                reasons.append("Does not allow 2.40 m x 2.40 m square (approx)")

            results.append({
                "space_name": raw_name,
                "area_m2": float(area),
                "min_required_m2": float(min_area),
                "clearance_ok": bool(clearance_ok),
                "result": "pass" if passed else "fail",
                "reason": "; ".join(reasons) if reasons else "Complies"
            })

    return results

# --------------------------
# Tool entrypoint (for an LLM router later)
# --------------------------
def living_room_compliance_tool(ifc_model_path: str):
    return living_room_compliance(ifc_model_path)

# --------------------------
# Schema (no API key needed)
# --------------------------
LIVING_ROOM_COMPLIANCE_SCHEMA = {
    "name": "living_room_compliance_tool",
    "description": "Checks living room spaces in an IFC for minimum area and clearance rules and returns per-space compliance results.",
    "parameters": {
        "type": "object",
        "properties": {
            "ifc_model_path": {
                "type": "string",
                "description": "Filesystem path to the IFC model."
            }
        },
        "required": ["ifc_model_path"]
    }
}

# --------------------------
# Usage example (prints schema + report)
# --------------------------
if __name__ == "__main__":
    print("Schema OK:")
    print(json.dumps(LIVING_ROOM_COMPLIANCE_SCHEMA, indent=2))

    ifc_path = "/content/ifc-bench/projects/duplex/arc.ifc"  # change if needed
    report = living_room_compliance_tool(ifc_path)

    print("\nReport:")
    if not report:
        print("No living room spaces matched (keyword 'living') or no computable areas.")
    else:
        for r in report:
            print(f"{r['space_name']}: {r['area_m2']:.2f} m² (min {r['min_required_m2']:.2f} m²) → {r['result']}")
            if r["result"] == "fail":
                print(f"  Reason: {r['reason']}")

Schema OK:
{
  "name": "living_room_compliance_tool",
  "description": "Checks living room spaces in an IFC for minimum area and clearance rules and returns per-space compliance results.",
  "parameters": {
    "type": "object",
    "properties": {
      "ifc_model_path": {
        "type": "string",
        "description": "Filesystem path to the IFC model."
      }
    },
    "required": [
      "ifc_model_path"
    ]
  }
}

Report:
Living Room: 109.86 m² (min 10.00 m²) → pass
Living Room: 109.86 m² (min 10.00 m²) → pass


# Tool D Service Area Heights

In [9]:
import json
import ifcopenshell
import ifcopenshell.geom

# --------------------------
# Geometry settings
# --------------------------
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

# --------------------------
# Helper functions
# --------------------------
def load_model(ifc_model_path):
    return ifcopenshell.open(ifc_model_path)

def get_space_name(space):
    """Get descriptive name if available, fallback to LongName/Name/GlobalId."""
    if getattr(space, "LongName", None):
        return str(space.LongName)
    if getattr(space, "Name", None):
        return str(space.Name)
    return str(getattr(space, "GlobalId", "Unnamed"))

def get_space_height(space):
    """
    Approximate height of a space from geometry bounding box.
    Uses Z coordinates of all vertices.
    """
    try:
        shape = ifcopenshell.geom.create_shape(settings, space)
        verts = shape.geometry.verts
        if not verts:
            return 0.0
        zs = verts[2::3]
        return float(max(zs) - min(zs))
    except Exception:
        return 0.0

def is_service_space(space_name_lower):
    """Bathrooms, kitchens, hallways by keywords."""
    service_keywords = [
        "bath", "bathroom", "baño", "bano", "wc", "toilet",
        "kitchen", "cocina",
        "hall", "hallway", "corridor", "pasillo"
    ]
    return any(k in space_name_lower for k in service_keywords)

# --------------------------
# Compliance check
# --------------------------
def service_spaces_min_height_check(ifc_model_path, min_height=2.20):
    """
    Regulation:
      - Minimum height in bathrooms, kitchens, and hallways is 2.20 m

    Returns:
      - result: 'pass' or 'fail'
      - reason
      - room_heights: dict label -> height
      - checked_spaces: list of labels that were evaluated
    """
    model = load_model(ifc_model_path)
    spaces = model.by_type("IfcSpace")

    room_heights = {}
    checked_spaces = []

    for space in spaces:
        label = get_space_name(space)
        label_l = label.lower()

        if not is_service_space(label_l):
            continue

        checked_spaces.append(label)
        h = get_space_height(space)
        room_heights[label] = h

        if h < float(min_height):
            return {
                "result": "fail",
                "reason": f"{label} height below {float(min_height):.2f}m",
                "room_heights": room_heights,
                "checked_spaces": checked_spaces
            }

    if not checked_spaces:
        return {
            "result": "fail",
            "reason": "No bathrooms/kitchens/hallways matched by keywords, nothing checked",
            "room_heights": room_heights,
            "checked_spaces": checked_spaces
        }

    return {
        "result": "pass",
        "reason": f"All checked bathrooms/kitchens/hallways meet minimum height {float(min_height):.2f}m",
        "room_heights": room_heights,
        "checked_spaces": checked_spaces
    }

# --------------------------
# Tool entrypoint (for an LLM router later)
# --------------------------
def service_spaces_min_height_check_tool(ifc_model_path: str, min_height: float = 2.20):
    return service_spaces_min_height_check(ifc_model_path, min_height)

# --------------------------
# Schema (no API key needed)
# --------------------------
SERVICE_SPACES_MIN_HEIGHT_SCHEMA = {
    "name": "service_spaces_min_height_check_tool",
    "description": "Checks bathrooms, kitchens, and hallways in an IFC for a minimum height requirement (default 2.20m).",
    "parameters": {
        "type": "object",
        "properties": {
            "ifc_model_path": {
                "type": "string",
                "description": "Filesystem path to the IFC model."
            },
            "min_height": {
                "type": "number",
                "description": "Minimum height in meters. Default is 2.20."
            }
        },
        "required": ["ifc_model_path"]
    }
}

# --------------------------
# Usage example (prints schema + pass/fail)
# --------------------------
if __name__ == "__main__":
    print("Schema OK:")
    print(json.dumps(SERVICE_SPACES_MIN_HEIGHT_SCHEMA, indent=2))

    ifc_path = "/content/ifc-bench/projects/duplex/arc.ifc"  # change if needed
    check = service_spaces_min_height_check_tool(ifc_path, min_height=2.20)

    print("\nCheck result:")
    print(check["result"])
    print(check["reason"])
    print("Room heights:", check["room_heights"])
    print("Checked spaces:", check["checked_spaces"])

Schema OK:
{
  "name": "service_spaces_min_height_check_tool",
  "description": "Checks bathrooms, kitchens, and hallways in an IFC for a minimum height requirement (default 2.20m).",
  "parameters": {
    "type": "object",
    "properties": {
      "ifc_model_path": {
        "type": "string",
        "description": "Filesystem path to the IFC model."
      },
      "min_height": {
        "type": "number",
        "description": "Minimum height in meters. Default is 2.20."
      }
    },
    "required": [
      "ifc_model_path"
    ]
  }
}

Check result:
pass
All checked bathrooms/kitchens/hallways meet minimum height 2.20m
Room heights: {'Hallway': 2.8810000000001947, 'Bathroom 2': 2.5870000000001836, 'Bathroom 1': 2.587000000000001, 'Kitchen': 2.587000000000001}
Checked spaces: ['Hallway', 'Hallway', 'Bathroom 2', 'Bathroom 1', 'Kitchen', 'Kitchen', 'Bathroom 2', 'Bathroom 1']
